In [53]:
import sys
import os
path = os.path.abspath('')
path = os.path.dirname(path)
sys.path.insert(1, str(path))


In [54]:

from datetime import date
import os
import load_csv

data_frame = load_csv.load("../data/clean_data.csv")
context = load_csv.getContext()

In [55]:
data_frame.show()

+--------------------+----------------+-----------------+----------+--------------------+-------------------+----------------+---+------+----------+--------------------+----------------+----------------------+----------------+-------------------+
|            order_id|     customer_id|    customer_name|product_id|        product_name|   product_category|    payment_type|qty| price|  datetime|             country|            city|ecommerce_website_name|  payment_txn_id|payment_txn_success|
+--------------------+----------------+-----------------+----------+--------------------+-------------------+----------------+---+------+----------+--------------------+----------------+----------------------+----------------+-------------------+
|2NEBnZETy1FdFpKJf...|wEwuo1GDdnbC8ju5|   STEPHEN THOMAS|     14315|ESR Boost Kicksta...|tv, audio & cameras|             UPI|  3| 25.19|2022-05-19|       United States|       Blackhawk|       www.wayfair.com|XXFqXfizoOtsQ0S1|                  Y|
|cs4mesS4QiK

In [56]:
data_frame.createOrReplaceTempView('orders')

In [97]:
def get_popularity_over_year():
    query_split_datetime = 'CONCAT(YEAR(datetime), \'_Q\',QUARTER(datetime)) AS year_quarter'
    query_quarters = f'SELECT {query_split_datetime} FROM orders'
    query_unique_quarters = f'SELECT DISTINCT year_quarter FROM ({query_quarters}) ORDER BY year_quarter'
    quarters_df = context.sql(query_unique_quarters)
    quarter_list = quarters_df.collect()
    quarter_list = [row['year_quarter'] for row in quarter_list]

    query_pivot = '('
    for index, quarter in enumerate(quarter_list):
        if index == len(quarter_list) - 1:
            query_pivot += f'\'{quarter}\')'
        else:
            query_pivot += f'\'{quarter}\', '

    query_popularity_over_year_subquery = f'SELECT {query_split_datetime}, product_name, qty FROM orders'
    query_popularity_over_year = f'SELECT * FROM ({query_popularity_over_year_subquery}) PIVOT (SUM(qty) AS popularity FOR year_quarter IN {query_pivot})'

    pop_year_df = context.sql(query_popularity_over_year)
    pop_year_df = pop_year_df.na.fill(0)
    pop_year_df.show(1000)
    return pop_year_df

def get_popularity_over_country():
    query_unique_countries = f'SELECT DISTINCT country FROM orders ORDER BY country'
    countries_df = context.sql(query_unique_countries)
    country_list = countries_df.collect()
    country_list = [row['country'] for row in country_list]

    query_pivot = '('
    for index, country in enumerate(country_list):
        if index == len(country_list) - 1:
            query_pivot += f'\'{country}\')'
        else:
            query_pivot += f'\'{country}\', '

    query_popularity_over_country_subquery = f'SELECT country, product_name, qty FROM orders'
    query_popularity_over_country = f'SELECT * FROM ({query_popularity_over_country_subquery}) PIVOT (SUM(qty) AS popularity FOR country IN {query_pivot})'

    pop_country_df = context.sql(query_popularity_over_country)
    pop_country_df = pop_country_df.na.fill(0)
    pop_country_df.show(1000)
    return pop_country_df

pop_year_df = get_popularity_over_year()
pop_country_df = get_popularity_over_country()

+--------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|        product_name|2022_Q1|2022_Q2|2022_Q3|2022_Q4|2023_Q1|2023_Q2|2023_Q3|2023_Q4|2024_Q1|2024_Q2|2024_Q3|
+--------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|G4Girl Women's Re...|      0|      0|      5|      0|      4|      7|     16|      2|      9|      5|      4|
|Priyaasi American...|      0|      0|      0|      0|     10|      6|      7|     18|      0|      0|      0|
|Zeel Clothing Wom...|      2|      8|     16|      0|      0|      5|     14|     17|     15|     19|     10|
|The Indian Garage...|      0|     11|      4|     13|      5|     11|      0|     12|      1|     32|      5|
|Pixievixen Boy's ...|      0|     32|     32|     12|     10|      0|      2|     22|     16|      7|      7|
|3NH® 2Pcs Strolle...|     10|      8|      6|      0|      5|      0|      0|     17|      0|      2|      0|
|

24/08/07 15:19:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+---------+---------+------+------+------+----+-----+-------+-------+------+-----+-------------+--------+-----+---------+----+----+-----+----------+-----+----+------+-------+------+----+-----------+------+----------------------------------------+---------+------+--------+------+-------+-------------+----------+--------+
|        product_name|Argentina|Australia|Bhutan|Brazil|Canada|Chad|China|Czechia|Finland|France|Gabon|Guinea-Bissau|Honduras|India|Indonesia|Iran|Iraq|Japan|Kazakhstan|Kenya|Mali|Mexico|Nigeria|Panama|Peru|Puerto Rico|Russia|South Georgia And South Sandwich Islands|Sri Lanka|Sweden|Thailand|Turkey|Ukraine|United States|Uzbekistan|Zimbabwe|
+--------------------+---------+---------+------+------+------+----+-----+-------+-------+------+-----+-------------+--------+-----+---------+----+----+-----+----------+-----+----+------+-------+------+----+-----------+------+----------------------------------------+---------+------+--------+------+-------+------

In [98]:

# for jupyter
path1 = path + "/data/miguel/popularity_over_year.csv"
path2 = path + "/data/miguel/popularity_over_country.csv"
pop_year_df.toPandas().to_csv(str(path1), index=False)
pop_country_df.toPandas().to_csv(str(path2), index=False)